In [1]:
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from skimage.metrics import mean_squared_error, structural_similarity, peak_signal_noise_ratio

import torch
import torch.nn as nn
import torch.optim as optim

from utils import check_torch, pix2vid_dataset, DualCustomLoss, DualLpLoss, NeuralPix2Vid

In [3]:
NR, NT = 1272, 60
NX, NY = 64, 64
milli  = 1e-3
mega   = 1e6
Darcy  = 9.869233e-13
psi2pa = 6894.75729
co2rho = 686.5400
sec2yr = 1/(3600*24*365.25)
device = check_torch()

device = torch.device('cpu')

timesteps = np.load('timesteps.npy')
deltaTime = np.concatenate([np.ones((30)), np.ones((30))*50])
print('timesteps: {} | deltaT: {}'.format(len(timesteps), np.unique(deltaTime)))
(Xt, ct, y1t, y2t, all_volumes, idx), (trainloader, validloader) = pix2vid_dataset(send_to_device=True, device=device)

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.3.1.post300 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3090
Torch device: cuda
------------------------------------------------------------
timesteps: 60 | deltaT: [ 1. 50.]
Train - X:  torch.Size([1000, 4, 64, 64])     | c:  torch.Size([1000, 30, 5])
        y1: torch.Size([1000, 30, 2, 64, 64]) | y2: torch.Size([1000, 30, 1, 64, 64])
--------------------
Valid - X:  torch.Size([136, 4, 64, 64])     | c:  torch.Size([136, 30, 5])
        y1: torch.Size([136, 30, 2, 64, 64]) | y2: torch.Size([136, 30, 1, 64, 64])
--------------------
Test  - X:  torch.Size([136, 4, 64, 64])     | c:  torch.Size([136, 30, 5])
        y1: torch.Size([136, 30, 2, 64, 64]) | y2: torch.Size([136, 30, 1, 64, 64])


In [4]:
model = NeuralPix2Vid(device=device).to(device)
nparams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('# parameters: {:,} | device: {}'.format(nparams, model.device))
criterion = DualLpLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# parameters: 4,261,815 | device: cpu


In [5]:
start = time()
epochs, monitor = 10, 1
train_loss, valid_loss = [], []
for epoch in range(epochs):
    epoch_train_loss = []
    model.train()
    for i, (x,c,y1,y2) in enumerate(trainloader):
        optimizer.zero_grad()
        u1, u2 = model(x,c)
        loss = criterion(y1, y2, u1, u2)
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())
    train_loss.append(np.mean(epoch_train_loss))
    # validation
    model.eval()
    epoch_valid_loss = []
    with torch.no_grad():
        for i, (x,c,y1,y2) in enumerate(validloader):
            u1, u2 = model(x,c)
            loss = criterion(y1, y2, u1, u2)
            epoch_valid_loss.append(loss.item())
    valid_loss.append(np.mean(epoch_valid_loss))
    # progress
    if (epoch+1) % monitor == 0:
        print('Epoch: [{}/{}] | Train Loss: {:.5f} | Valid Loss: {:.5f}'.format(epoch+1, epochs, train_loss[-1], valid_loss[-1]))

print('Total training time: {:.3f} minuts'.format((time()-start)/60))
torch.save(model.state_dict(), 'model.pth')
losses = pd.DataFrame({'train': train_loss, 'valid': valid_loss})
losses.to_csv('losses.csv', index=False)

KeyboardInterrupt: 